In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv1D, MaxPooling1D, Dropout, Flatten, Lambda
from tensorflow.keras.optimizers import Adam  
from sklearn.model_selection import train_test_split



In [2]:
def load_data_from_folders(base_folder):
    data = []
    labels = []
    ground_truths = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    
    for i, ground_truth in enumerate(ground_truths):
        folder_path = os.path.join(base_folder, f'class_{i}')
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r') as file:
                    
                    numbers = [float(num) for line in file for num in line.split(',')]
                    if len(numbers) == 507:  
                        data.append(numbers)
                        labels.append(ground_truth)
    return np.array(data), np.array(labels)



In [3]:
base_folder = 'B:\jupyter_notebook\localization'
data, labels = load_data_from_folders(base_folder)


In [4]:
index_of_30 = np.where(labels == 30)[0]  
if len(index_of_30) > 0:
    print(f"30度的第一个样本数据: {data[index_of_30[0]]}")
else:
    print("没有找到30度的样本")

30度的第一个样本数据: [ 1.3096e+02  2.0320e+01  1.9100e+00 -6.6300e+00 -2.3700e+00  4.0600e+00
  2.8600e+00 -2.3800e+00 -3.0900e+00  1.4700e+00  2.8100e+00 -8.4000e-01
 -2.4900e+00  1.3085e+02  1.9610e+01  1.5100e+00 -7.7500e+00 -2.3500e+00
  3.6400e+00  2.3200e+00 -1.8700e+00 -2.7800e+00  1.3900e+00  3.1600e+00
 -5.1000e-01 -2.0900e+00  1.2953e+02  1.9610e+01  2.1100e+00 -6.8600e+00
 -2.0500e+00  3.8800e+00  2.6200e+00 -2.6400e+00 -3.0800e+00  1.5400e+00
  2.8200e+00 -9.3000e-01 -2.5000e+00  1.2978e+02  1.9630e+01  9.0000e-02
 -7.9600e+00 -3.1100e+00  3.5000e+00  2.6600e+00 -2.4300e+00 -2.4200e+00
  1.6700e+00  3.3100e+00 -4.5000e-01 -2.7000e+00  1.2931e+02  1.9570e+01
  8.4000e-01 -6.8100e+00 -2.3200e+00  3.0500e+00  1.7400e+00 -2.4500e+00
 -2.1900e+00  1.9800e+00  3.0100e+00 -8.8000e-01 -2.5400e+00  1.2875e+02
  1.9900e+01  4.9000e-01 -7.3900e+00 -2.1700e+00  3.4700e+00  2.7100e+00
 -2.5400e+00 -2.2500e+00  1.1900e+00  2.8900e+00 -6.3000e-01 -2.4300e+00
  1.2873e+02  1.9470e+01  1.0600e+00 -

In [5]:

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:

BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(buffer_size=len(train_data)).batch(BATCH_SIZE)
validation_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(BATCH_SIZE)


input_length = 507  

In [11]:
def cyclic_mse(y_true, y_pred):
    pi = tf.constant(np.pi, dtype=tf.float32)
    deg = tf.constant(180.0, dtype=tf.float32)  
    y_true = tf.cast(y_true, dtype=tf.float32)  
    y_pred = tf.cast(y_pred, dtype=tf.float32)  #y_pred to float32
    return tf.reduce_mean(tf.square(
        tf.atan2(
            tf.sin(y_true * pi / deg - y_pred * pi / deg),
            tf.cos(y_true * pi / deg - y_pred * pi / deg)
        )
    ) * deg / pi)


In [12]:
model = Sequential()
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1))  
model.add(tf.keras.layers.Lambda(lambda x: x % 360))  

In [13]:
LEARNING_RATE = 0.005
opt = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
model.compile(loss=cyclic_mse, optimizer=opt, metrics=[cyclic_mse])

In [14]:

EPOCHS = 300
model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=2)
# the following printed results are just for testing

Epoch 1/300
3/3 - 1s - 350ms/step - cyclic_mse: 187.9126 - loss: 190.6431 - val_cyclic_mse: 191.9204 - val_loss: 191.9204
Epoch 2/300
3/3 - 0s - 9ms/step - cyclic_mse: 186.8439 - loss: 200.3309 - val_cyclic_mse: 198.4471 - val_loss: 198.4471
Epoch 3/300
3/3 - 0s - 7ms/step - cyclic_mse: 186.0153 - loss: 191.9742 - val_cyclic_mse: 200.0546 - val_loss: 200.0546
Epoch 4/300
3/3 - 0s - 9ms/step - cyclic_mse: 184.6907 - loss: 179.9753 - val_cyclic_mse: 204.6755 - val_loss: 204.6755
Epoch 5/300
3/3 - 0s - 7ms/step - cyclic_mse: 183.0047 - loss: 190.2693 - val_cyclic_mse: 212.6091 - val_loss: 212.6091
Epoch 6/300
3/3 - 0s - 10ms/step - cyclic_mse: 182.7997 - loss: 184.1008 - val_cyclic_mse: 210.0609 - val_loss: 210.0609
Epoch 7/300
3/3 - 0s - 9ms/step - cyclic_mse: 185.8406 - loss: 190.4482 - val_cyclic_mse: 208.3343 - val_loss: 208.3343
Epoch 8/300
3/3 - 0s - 9ms/step - cyclic_mse: 183.3654 - loss: 174.2905 - val_cyclic_mse: 208.1041 - val_loss: 208.1041
Epoch 9/300
3/3 - 0s - 6ms/step - cyc

In [15]:

model.save('B:/jupyter_notebook/localization/local_model.h5')


In [16]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)                  │ (None, 39, 13)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 39, 8)               │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 20, 16)              │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 160)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             161 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,645 (10.34 KB)

 Trainable params: 881 (3.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,764 (6.89 KB)